This script will load the csv file of clean information
aggregate if needed (to get $_per_sqFT, a value neede for intensity of heatmap)
insert each row as a document that will have the address as the key 
may need two databases for now

- one that has all of the addresses so they can be compiled into a list for the heat map 
- one that has addresses as keys to the lat/lng so that they can be used to make a marker.

can document.find_one(address_query) pull the document that has matching info to address?


In [55]:
#-------------------------------------------------------------------------------------------------------------------------#
# https://www.w3schools.com/python/python_mongodb_query.asp

def query_database():
    # pull number of elements in list, display for user, request name of DB to pull
    # return the input, db_query = string dtype
    
    dblist = client.list_database_names()

    list_len = len(dblist)
    print(f'{list_len} Databases Found ... ')
    print()
    for name in dblist:
        print(f'-{name}')
        print()
    print('------------------------')

    database_query = input("Which Database would you like to load? Use exact spelling.. ")
    if database_query not in dblist:
        raise ValueError(f'Database "{database_query}" Not Found.')
    else:
        print()
        print(f'Database "{database_query}" Successfully Loaded..')
        print()
        return database_query
    
#-------------------------------------------------------------------------------------------------------------------------#

def query_collections(database_query):
    # list all collections in DB
    # request db to display collections
    # return string of which collection user selected to view documents
    
    #### add back feature to method?
    
#     database = query_database()
    loop = True
    collection_list = client[database_query].list_collection_names()
    collection_dict = {}
    x = len(collection_list)
   
    
    print(f' {x} collection(s) found in "{database_query}"')
    print('------------------------')
    # print each collection in list i.e. DB
    for count, collection in enumerate(collection_list):
        
        collection_dict[f'{collection}'] = count
        
        print()
        print(f'{count}-{collection}')
        print()
    print('------------------------')    
    print()
    
   # keep asking the query if the user misspells the collection

    while loop == True:
        try:
            collection_query = input("Which collection would you like to load? ")

            if collection_query not in collection_list:
                raise ValueError("Collection not found, Try again.")
                
            else:
                print("Collection found, Loading Documents.")
                loop = False
                return collection_query
                

        except ValueError:
            print()
            print(f'Collection "{collection_query}" not found, Try again.')
            print()
#-------------------------------------------------------------------------------------------------------------------------#

def query_documents(database_query,collection_query):

    collection = client[database_query][collection_query]
    print("want to see all documents? enter 'all' in both inputs. ")
    
    query_key = input('Desired Field in documents to pull matching results? ')
    query_value = input('Desired Value? NOTE: this will not pull numbers unless edited ')
    
    

    # EXAMPLE ------------------
    # query = {"query_key": query_value}
    # --------------------------
    if query_value == 'all' or query_key == 'all':
        address_documents = collection.find()
    else:
        query = {query_key:query_value}

        address_documents = collection.find(query)


    doc_list = []
    for doc in address_documents:
        doc_list.append(doc)
    return doc_list

In [2]:
import os
import pandas as pd
import pymongo
os.getcwd()

'C:\\Users\\Alec\\Documents\\Programming_Resources\\Boot_Camp\\16_project03\\LOCAL\\IPYNB files'

In [3]:
# set up pymongo connection

# https://docs.mongodb.com/manual/reference/default-mongodb-port/

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# database = query_database()
# collection = query_collections(database)



documents = query_documents("jj_earnings","test")
display(documents)

In [4]:
data_path = '../clean_data/Clean_Home1.csv'

clean_data = pd.read_csv(data_path)
clean_data.head()

,Owner,Address 1,Address 2,City,State,Full Address,2021 Building Value,2021 Land Value,2021 Total Value,Land (Acre),Land (SQFT),Est Tax,Total SQFT,$/Building SQFT,$/Total Value SQFT
0,PIERSON GARY DON,"5403 S COUNTY ROAD 1140MIDLAND, TX 79706-7128",COUNTY RD 01140 5403 S,MIDLAND,TEXAS,"COUNTY RD 01140 5403 S MIDLAND, TEXAS",94160.0,12000.0,106160.0,1.00,43560.0,844.42,1316.0,71.6,80.7
1,PORTER STEPHEN RAY,"5401 S COUNTY RD 1140MIDLAND, TX 79706",COUNTY RD 01140 5401 S,MIDLAND,TEXAS,"COUNTY RD 01140 5401 S MIDLAND, TEXAS",55950.0,58920.0,114870.0,4.91,213880.0,1613.89,1032.0,54.2,111.3
2,PORTER AUDIE L,"5407 S COUNTY ROAD 1140MIDLAND, TX 79706-7128",COUNTY RD 01140 5407 S,MIDLAND,TEXAS,"COUNTY RD 01140 5407 S MIDLAND, TEXAS",117590.0,27240.0,144830.0,1.00,43560.0,1010.62,1592.0,73.9,91.0
3,STICKEL CALEB &,CHRISTINA ELAINE STICKEL4700 E COUNTY RD 140MI...,COUNTY RD 00140 4700 E,MIDLAND,TEXAS,"COUNTY RD 00140 4700 E MIDLAND, TEXAS",289130.0,48000.0,337130.0,1.00,43560.0,4159.64,2553.0,113.3,132.1
4,VALLEY VIEW COMMUNITY CENTER,"E F JONESP O BOX 3270MIDLAND, TX 79702",FM RD 01213 5502,MIDLAND,TEXAS,"FM RD 01213 5502 MIDLAND, TEXAS",69280.0,27240.0,96520.0,2.27,98881.0,0.00,1984.0,34.9,48.6


In [5]:
clean_data['Full Address'][0]

'COUNTY RD 01140   5403   S MIDLAND, TEXAS'

In [6]:
test_cell = clean_data['Address 2'][0]
clean_cell = test_cell.replace(' ',"_")
display(clean_cell)

'COUNTY_RD_01140___5403___S'

In [7]:
# result = (' '*number_of_spaces_desired).join(string_to_replace_spaces.split())


split_cell = (' '*1).join(test_cell.split())

split_cell


'COUNTY RD 01140 5403 S'

In [8]:
address_2 = clean_data['Address 2']
address_list = address_2.to_list()

clean_address_2_list = []

for row in range(len(address_list)):
    x = address_2[row]
    print(x)
    clean_address_2 = (' '*1).join(x.split())
    clean_address_2_list.append(clean_address_2)
    print(clean_address_2)
    print('--------------')
    

COUNTY RD 01140   5403   S
COUNTY RD 01140 5403 S
--------------
COUNTY RD 01140   5401   S
COUNTY RD 01140 5401 S
--------------
COUNTY RD 01140   5407   S
COUNTY RD 01140 5407 S
--------------
COUNTY RD 00140   4700   E
COUNTY RD 00140 4700 E
--------------
FM RD 01213   5502
FM RD 01213 5502
--------------
FM RD 01213   4015
FM RD 01213 4015
--------------
ECR 130   4001
ECR 130 4001
--------------
COUNTY RD 01270   9009   S
COUNTY RD 01270 9009 S
--------------
COUNTY RD 00174   9201   W
COUNTY RD 00174 9201 W
--------------
COUNTY RD 01270   8601   S
COUNTY RD 01270 8601 S
--------------
COUNTY RD 00170   9315   W
COUNTY RD 00170 9315 W
--------------
COUNTY RD 00174   9205   W
COUNTY RD 00174 9205 W
--------------
COUNTY RD 00174   9404   W
COUNTY RD 00174 9404 W
--------------
COUNTY RD 01135   4007   S
COUNTY RD 01135 4007 S
--------------
COUNTY RD 00140   2408   E
COUNTY RD 00140 2408 E
--------------
COUNTY RD 01140   4912   S
COUNTY RD 01140 4912 S
--------------
FM RD 0030

--------------
COLORADO ST   1508   S
COLORADO ST 1508 S
--------------
BELMONT ST   1511
BELMONT ST 1511
--------------
BELMONT ST   1507
BELMONT ST 1507
--------------
BELMONT ST   1503
BELMONT ST 1503
--------------
BELMONT ST   1501
BELMONT ST 1501
--------------
COLORADO ST   1600   S
COLORADO ST 1600 S
--------------
COLORADO ST   1602   S
COLORADO ST 1602 S
--------------
COLORADO ST   1604   S
COLORADO ST 1604 S
--------------
COLORADO ST   1608   S
COLORADO ST 1608 S
--------------
COLORADO ST   1610   S
COLORADO ST 1610 S
--------------
BELMONT ST   1609
BELMONT ST 1609
--------------
BELMONT ST   1607
BELMONT ST 1607
--------------
BELMONT ST   1605
BELMONT ST 1605
--------------
BELMONT ST   1601
BELMONT ST 1601
--------------
LORAINE ST   1600   S
LORAINE ST 1600 S
--------------
LORAINE ST   1602   S
LORAINE ST 1602 S
--------------
LORAINE ST   1604   S
LORAINE ST 1604 S
--------------
LORAINE ST   1606   S
LORAINE ST 1606 S
--------------
COLORADO ST   1611   S
COLORADO

--------------
PARKER AV   1416   E
PARKER AV 1416 E
--------------
NOBLES AV   1405   E
NOBLES AV 1405 E
--------------
NOBLES AV   1401   E
NOBLES AV 1401 E
--------------
NOBLES AV   1300   E
NOBLES AV 1300 E
--------------
NOBLES AV   1308   E
NOBLES AV 1308 E
--------------
NOBLES AV   1312   E
NOBLES AV 1312 E
--------------
NOBLES AV   1400   E
NOBLES AV 1400 E
--------------
NOBLES AV   1402   E
NOBLES AV 1402 E
--------------
NOBLES AV   1404   E
NOBLES AV 1404 E
--------------
FAIRPARK RD   2201
FAIRPARK RD 2201
--------------
FAIRPARK RD   2204
FAIRPARK RD 2204
--------------
CLOVERDALE RD   2312
CLOVERDALE RD 2312
--------------
CLOVERDALE RD   2310
CLOVERDALE RD 2310
--------------
FAIRGROUNDS RD   1116   S
FAIRGROUNDS RD 1116 S
--------------
CLOVERDALE RD   2310
CLOVERDALE RD 2310
--------------
FAIRGROUNDS RD   1110   S
FAIRGROUNDS RD 1110 S
--------------
WEBSTER ST   1201   S
WEBSTER ST 1201 S
--------------
CLOVERDALE RD   1910
CLOVERDALE RD 1910
--------------
CLOVE

--------------
TIMBER LN   4807
TIMBER LN 4807
--------------
TIMBER LN   4811
TIMBER LN 4811
--------------
TIMBER LN   4813
TIMBER LN 4813
--------------
TIMBER LN   4815
TIMBER LN 4815
--------------
TIMBER LN   4817
TIMBER LN 4817
--------------
TIMBER LN   4819
TIMBER LN 4819
--------------
OAKRIDGE DR   3601
OAKRIDGE DR 3601
--------------
OAKRIDGE DR   3603
OAKRIDGE DR 3603
--------------
OAKRIDGE DR   3605
OAKRIDGE DR 3605
--------------
OAKRIDGE DR   3607
OAKRIDGE DR 3607
--------------
OAKRIDGE DR   3609
OAKRIDGE DR 3609
--------------
OAKRIDGE DR   3611
OAKRIDGE DR 3611
--------------
APOLLO CT   3612
APOLLO CT 3612
--------------
APOLLO CT   3610
APOLLO CT 3610
--------------
APOLLO CT   3604
APOLLO CT 3604
--------------
APOLLO CT   3602
APOLLO CT 3602
--------------
APOLLO CT   3600
APOLLO CT 3600
--------------
APOLLO CT   3601
APOLLO CT 3601
--------------
APOLLO CT   3603
APOLLO CT 3603
--------------
APOLLO CT   3605
APOLLO CT 3605
--------------
APOLLO CT   3607
APOL

WALNUT LN 1608 E
--------------
WALNUT LN   1606   E
WALNUT LN 1606 E
--------------
WALNUT LN   1602   E
WALNUT LN 1602 E
--------------
WALNUT LN   1600   E
WALNUT LN 1600 E
--------------
MULBERRY LN   1601
MULBERRY LN 1601
--------------
WALNUT LN   1706   E
WALNUT LN 1706 E
--------------
WALNUT LN   1704   E
WALNUT LN 1704 E
--------------
WALNUT LN   1702   E
WALNUT LN 1702 E
--------------
WALNUT LN   1804   E
WALNUT LN 1804 E
--------------
MULBERRY LN   1801
MULBERRY LN 1801
--------------
TYLER ST   809   N
TYLER ST 809 N
--------------
TYLER ST   805   N
TYLER ST 805 N
--------------
TYLER ST   803   N
TYLER ST 803 N
--------------
TYLER ST   801   N
TYLER ST 801 N
--------------
LEE ST   800   N
LEE ST 800 N
--------------
LEE ST   802   N
LEE ST 802 N
--------------
LEE ST   804   N
LEE ST 804 N
--------------
LEE ST   806   N
LEE ST 806 N
--------------
LEE ST   808   N
LEE ST 808 N
--------------
LEE ST   810   N
LEE ST 810 N
--------------
MESQUITE LN   1202
MESQUITE L

--------------
MAXWELL DR   2501
MAXWELL DR 2501
--------------
MAXWELL DR   2503
MAXWELL DR 2503
--------------
MAXWELL DR   2505
MAXWELL DR 2505
--------------
MAXWELL DR   2507
MAXWELL DR 2507
--------------
MAXWELL DR   2509
MAXWELL DR 2509
--------------
MAXWELL DR   2511
MAXWELL DR 2511
--------------
MAXWELL DR   2513
MAXWELL DR 2513
--------------
MAXWELL DR   2601
MAXWELL DR 2601
--------------
MAXWELL DR   2603
MAXWELL DR 2603
--------------
MAXWELL DR   2605
MAXWELL DR 2605
--------------
MAXWELL DR   2607
MAXWELL DR 2607
--------------
MAXWELL DR   2609
MAXWELL DR 2609
--------------
MAXWELL DR   2611
MAXWELL DR 2611
--------------
MAXWELL DR   2613
MAXWELL DR 2613
--------------
MAXWELL DR   2615
MAXWELL DR 2615
--------------
MAXWELL DR   2415
MAXWELL DR 2415
--------------
MAXWELL DR   2413
MAXWELL DR 2413
--------------
MAXWELL DR   2409
MAXWELL DR 2409
--------------
MAXWELL DR   2407
MAXWELL DR 2407
--------------
MAXWELL DR   2405
MAXWELL DR 2405
--------------
MAXWE

In [9]:
clean_data['Address 2'] = clean_address_2_list
clean_data['Address 2']

0       COUNTY RD 01140 5403 S
1       COUNTY RD 01140 5401 S
2       COUNTY RD 01140 5407 S
3       COUNTY RD 00140 4700 E
4             FM RD 01213 5502
                 ...          
6429            FANNIN AV 2507
6430            FANNIN AV 2505
6431            FANNIN AV 2503
6432            FANNIN AV 2501
6433          WADLEY AV 2501 W
Name: Address 2, Length: 6434, dtype: object

In [10]:
clean_data

,Owner,Address 1,Address 2,City,State,Full Address,2021 Building Value,2021 Land Value,2021 Total Value,Land (Acre),Land (SQFT),Est Tax,Total SQFT,$/Building SQFT,$/Total Value SQFT
0,PIERSON GARY DON,"5403 S COUNTY ROAD 1140MIDLAND, TX 79706-7128",COUNTY RD 01140 5403 S,MIDLAND,TEXAS,"COUNTY RD 01140 5403 S MIDLAND, TEXAS",94160.0,12000.0,106160.0,1.000,43560.0,844.42,1316.0,71.6,80.7
1,PORTER STEPHEN RAY,"5401 S COUNTY RD 1140MIDLAND, TX 79706",COUNTY RD 01140 5401 S,MIDLAND,TEXAS,"COUNTY RD 01140 5401 S MIDLAND, TEXAS",55950.0,58920.0,114870.0,4.910,213880.0,1613.89,1032.0,54.2,111.3
2,PORTER AUDIE L,"5407 S COUNTY ROAD 1140MIDLAND, TX 79706-7128",COUNTY RD 01140 5407 S,MIDLAND,TEXAS,"COUNTY RD 01140 5407 S MIDLAND, TEXAS",117590.0,27240.0,144830.0,1.000,43560.0,1010.62,1592.0,73.9,91.0
3,STICKEL CALEB &,CHRISTINA ELAINE STICKEL4700 E COUNTY RD 140MI...,COUNTY RD 00140 4700 E,MIDLAND,TEXAS,"COUNTY RD 00140 4700 E MIDLAND, TEXAS",289130.0,48000.0,337130.0,1.000,43560.0,4159.64,2553.0,113.3,132.1
4,VALLEY VIEW COMMUNITY CENTER,"E F JONESP O BOX 3270MIDLAND, TX 79702",FM RD 01213 5502,MIDLAND,TEXAS,"FM RD 01213 5502 MIDLAND, TEXAS",69280.0,27240.0,96520.0,2.270,98881.0,0.00,1984.0,34.9,48.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429,MC CLAINE BRIAN K,"2507 FANNIN AVEMIDLAND, TX 79705-7311",FANNIN AV 2507,MIDLAND,TEXAS,"FANNIN AV 2507 MIDLAND, TEXAS",227720.0,24770.0,252490.0,0.253,11008.0,3873.71,2139.0,106.5,118.0
6430,CROSS THOMAS C,"2505 FANNIN AVEMIDLAND, TX 79705-7311",FANNIN AV 2505,MIDLAND,TEXAS,"FANNIN AV 2505 MIDLAND, TEXAS",236320.0,23900.0,260220.0,0.244,10624.0,4000.13,2193.0,107.8,118.7
6431,WEAVER ANDREW R,"2503 FANNIN AVMIDLAND, TX 79705",FANNIN AV 2503,MIDLAND,TEXAS,"FANNIN AV 2503 MIDLAND, TEXAS",241760.0,23130.0,264890.0,0.236,10280.0,4076.50,1992.0,121.4,133.0
6432,MELTON TOBY,"2501 FANNIN AVMIDLAND, TX 79705",FANNIN AV 2501,MIDLAND,TEXAS,"FANNIN AV 2501 MIDLAND, TEXAS",311570.0,26210.0,337780.0,0.267,11648.0,5268.64,3262.0,95.5,103.5


In [11]:
test_cell = clean_data['Address 1'][0]
clean_cell = test_cell.replace(' ',"_")
display(clean_cell)

'5403_S_COUNTY_ROAD_1140MIDLAND,_TX_79706-7128'

In [43]:
# declare Database
db = client.midland_property_info

# declare collection(s)

# collection for the results of the main search page, PROPERTY ID search
addresses = db.addresses

# collection for the "view_property_results" link on main search results page
# view_property_results = db.view_property_results


In [44]:
clean_data = clean_data.rename(columns ={
    'Owner':'owner',
    'Address 1':'address_1',
    'Address 2':'address_2',
    'City':'city',
    'State':'state',
    'Full Address':'full_address',
    '2021 Building Value':'2021_building_value',
    '2021 Land Value':'2021_land_value',
    '2021 Total Value':'2021_total_value',
    'Land (Acre)':'land_acre',
    'Land (SQFT)':'land_sqft',
    'Est Tax':'est_tax',
    'Total SQFT':'total_sqft',
    '$/Building SQFT':'$_per_building_sqft',
    '$/Total Value SQFT':'$_per_total_value_sqft'
})

In [45]:
clean_data

,owner,address_1,address_2,city,state,full_address,2021_building_value,2021_land_value,2021_total_value,land_acre,land_sqft,est_tax,total_sqft,$_per_building_sqft,$_per_total_value_sqft
0,PIERSON GARY DON,"5403 S COUNTY ROAD 1140MIDLAND, TX 79706-7128",COUNTY RD 01140 5403 S,MIDLAND,TEXAS,"COUNTY RD 01140 5403 S MIDLAND, TEXAS",94160.0,12000.0,106160.0,1.000,43560.0,844.42,1316.0,71.6,80.7
1,PORTER STEPHEN RAY,"5401 S COUNTY RD 1140MIDLAND, TX 79706",COUNTY RD 01140 5401 S,MIDLAND,TEXAS,"COUNTY RD 01140 5401 S MIDLAND, TEXAS",55950.0,58920.0,114870.0,4.910,213880.0,1613.89,1032.0,54.2,111.3
2,PORTER AUDIE L,"5407 S COUNTY ROAD 1140MIDLAND, TX 79706-7128",COUNTY RD 01140 5407 S,MIDLAND,TEXAS,"COUNTY RD 01140 5407 S MIDLAND, TEXAS",117590.0,27240.0,144830.0,1.000,43560.0,1010.62,1592.0,73.9,91.0
3,STICKEL CALEB &,CHRISTINA ELAINE STICKEL4700 E COUNTY RD 140MI...,COUNTY RD 00140 4700 E,MIDLAND,TEXAS,"COUNTY RD 00140 4700 E MIDLAND, TEXAS",289130.0,48000.0,337130.0,1.000,43560.0,4159.64,2553.0,113.3,132.1
4,VALLEY VIEW COMMUNITY CENTER,"E F JONESP O BOX 3270MIDLAND, TX 79702",FM RD 01213 5502,MIDLAND,TEXAS,"FM RD 01213 5502 MIDLAND, TEXAS",69280.0,27240.0,96520.0,2.270,98881.0,0.00,1984.0,34.9,48.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429,MC CLAINE BRIAN K,"2507 FANNIN AVEMIDLAND, TX 79705-7311",FANNIN AV 2507,MIDLAND,TEXAS,"FANNIN AV 2507 MIDLAND, TEXAS",227720.0,24770.0,252490.0,0.253,11008.0,3873.71,2139.0,106.5,118.0
6430,CROSS THOMAS C,"2505 FANNIN AVEMIDLAND, TX 79705-7311",FANNIN AV 2505,MIDLAND,TEXAS,"FANNIN AV 2505 MIDLAND, TEXAS",236320.0,23900.0,260220.0,0.244,10624.0,4000.13,2193.0,107.8,118.7
6431,WEAVER ANDREW R,"2503 FANNIN AVMIDLAND, TX 79705",FANNIN AV 2503,MIDLAND,TEXAS,"FANNIN AV 2503 MIDLAND, TEXAS",241760.0,23130.0,264890.0,0.236,10280.0,4076.50,1992.0,121.4,133.0
6432,MELTON TOBY,"2501 FANNIN AVMIDLAND, TX 79705",FANNIN AV 2501,MIDLAND,TEXAS,"FANNIN AV 2501 MIDLAND, TEXAS",311570.0,26210.0,337780.0,0.267,11648.0,5268.64,3262.0,95.5,103.5


In [46]:
data_no_dupes = clean_data.drop_duplicates()

In [47]:
index = data_no_dupes.index.to_list()
index_length = len(index)
index_length

6433

In [48]:
row = data_no_dupes.iloc[[0]]
row_dict = row.to_dict()
row_dict

{'owner': {0: 'PIERSON GARY DON'},
 'address_1': {0: '5403 S COUNTY ROAD 1140MIDLAND, TX 79706-7128'},
 'address_2': {0: 'COUNTY RD 01140 5403 S'},
 'city': {0: 'MIDLAND'},
 'state': {0: 'TEXAS'},
 'full_address': {0: 'COUNTY RD 01140   5403   S MIDLAND, TEXAS'},
 '2021_building_value': {0: 94160.0},
 '2021_land_value': {0: 12000.0},
 '2021_total_value': {0: 106160.0},
 'land_acre': {0: 1.0},
 'land_sqft': {0: 43560.0},
 'est_tax': {0: 844.42},
 'total_sqft': {0: 1316.0},
 '$_per_building_sqft': {0: 71.6},
 '$_per_total_value_sqft': {0: 80.7}}

In [49]:
for i in index:
    # row = row of index value, i
    row = data_no_dupes.iloc[[i]]
    # turns row of df into dictionary to allow to inserting to MongoDB
    row_dict = row.to_dict()
    clean_doc = {}
    clean_entry = {}
    # edit the row to equal only {key:value} NOT {key:{key_2: value}}
    clean_doc['owner']= row_dict['owner'][i]
    clean_doc['address_1']= row_dict['address_1'][i]
    clean_doc['address_2']= row_dict['address_2'][i]
    clean_doc['city']= row_dict['city'][i]
    clean_doc['state']= row_dict['state'][i]
    clean_doc['full_address']= row_dict['full_address'][i]
    clean_doc['2021_building_value']= row_dict['2021_building_value'][i]
    clean_doc['2021_land_value']= row_dict['2021_land_value'][i]
    clean_doc['2021_total_value']= row_dict['2021_total_value'][i]
    clean_doc['land_acre']= row_dict['land_acre'][i]
    clean_doc['land_sqft']= row_dict['land_sqft'][i]
    clean_doc['est_tax']= row_dict['est_tax'][i]
    clean_doc['total_sqft']= row_dict['total_sqft'][i]
    clean_doc['$_per_building_sqft']= row_dict['$_per_building_sqft'][i]
    clean_doc['$_per_total_value_sqft']= row_dict['$_per_total_value_sqft'][i]
    
#     key = row_dict['address_2'][i]

    
#     clean_entry[key]= clean_doc
    addresses.insert_one(clean_doc)
    print()
    print(f'Row {i} saved to "bulk_results" in MongoDB')
    print('-----------------------------------')


Row 0 saved to "bulk_results" in MongoDB
-----------------------------------

Row 1 saved to "bulk_results" in MongoDB
-----------------------------------

Row 2 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4 saved to "bulk_results" in MongoDB
-----------------------------------

Row 5 saved to "bulk_results" in MongoDB
-----------------------------------

Row 6 saved to "bulk_results" in MongoDB
-----------------------------------

Row 7 saved to "bulk_results" in MongoDB
-----------------------------------

Row 8 saved to "bulk_results" in MongoDB
-----------------------------------

Row 9 saved to "bulk_results" in MongoDB
-----------------------------------

Row 10 saved to "bulk_results" in MongoDB
-----------------------------------

Row 11 saved to "bulk_results" in MongoDB
-----------------------------------

Row 12 saved to "bulk_results" in MongoDB
-------------------


Row 157 saved to "bulk_results" in MongoDB
-----------------------------------

Row 158 saved to "bulk_results" in MongoDB
-----------------------------------

Row 159 saved to "bulk_results" in MongoDB
-----------------------------------

Row 160 saved to "bulk_results" in MongoDB
-----------------------------------

Row 161 saved to "bulk_results" in MongoDB
-----------------------------------

Row 162 saved to "bulk_results" in MongoDB
-----------------------------------

Row 163 saved to "bulk_results" in MongoDB
-----------------------------------

Row 164 saved to "bulk_results" in MongoDB
-----------------------------------

Row 165 saved to "bulk_results" in MongoDB
-----------------------------------

Row 166 saved to "bulk_results" in MongoDB
-----------------------------------

Row 167 saved to "bulk_results" in MongoDB
-----------------------------------

Row 168 saved to "bulk_results" in MongoDB
-----------------------------------

Row 169 saved to "bulk_results" in Mong


Row 321 saved to "bulk_results" in MongoDB
-----------------------------------

Row 322 saved to "bulk_results" in MongoDB
-----------------------------------

Row 323 saved to "bulk_results" in MongoDB
-----------------------------------

Row 324 saved to "bulk_results" in MongoDB
-----------------------------------

Row 325 saved to "bulk_results" in MongoDB
-----------------------------------

Row 326 saved to "bulk_results" in MongoDB
-----------------------------------

Row 327 saved to "bulk_results" in MongoDB
-----------------------------------

Row 328 saved to "bulk_results" in MongoDB
-----------------------------------

Row 329 saved to "bulk_results" in MongoDB
-----------------------------------

Row 330 saved to "bulk_results" in MongoDB
-----------------------------------

Row 331 saved to "bulk_results" in MongoDB
-----------------------------------

Row 332 saved to "bulk_results" in MongoDB
-----------------------------------

Row 333 saved to "bulk_results" in Mong


Row 480 saved to "bulk_results" in MongoDB
-----------------------------------

Row 481 saved to "bulk_results" in MongoDB
-----------------------------------

Row 482 saved to "bulk_results" in MongoDB
-----------------------------------

Row 483 saved to "bulk_results" in MongoDB
-----------------------------------

Row 484 saved to "bulk_results" in MongoDB
-----------------------------------

Row 485 saved to "bulk_results" in MongoDB
-----------------------------------

Row 486 saved to "bulk_results" in MongoDB
-----------------------------------

Row 487 saved to "bulk_results" in MongoDB
-----------------------------------

Row 488 saved to "bulk_results" in MongoDB
-----------------------------------

Row 489 saved to "bulk_results" in MongoDB
-----------------------------------

Row 490 saved to "bulk_results" in MongoDB
-----------------------------------

Row 491 saved to "bulk_results" in MongoDB
-----------------------------------

Row 492 saved to "bulk_results" in Mong


Row 646 saved to "bulk_results" in MongoDB
-----------------------------------

Row 647 saved to "bulk_results" in MongoDB
-----------------------------------

Row 648 saved to "bulk_results" in MongoDB
-----------------------------------

Row 649 saved to "bulk_results" in MongoDB
-----------------------------------

Row 650 saved to "bulk_results" in MongoDB
-----------------------------------

Row 651 saved to "bulk_results" in MongoDB
-----------------------------------

Row 652 saved to "bulk_results" in MongoDB
-----------------------------------

Row 653 saved to "bulk_results" in MongoDB
-----------------------------------

Row 654 saved to "bulk_results" in MongoDB
-----------------------------------

Row 655 saved to "bulk_results" in MongoDB
-----------------------------------

Row 656 saved to "bulk_results" in MongoDB
-----------------------------------

Row 657 saved to "bulk_results" in MongoDB
-----------------------------------

Row 658 saved to "bulk_results" in Mong

...


Row 3771 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3772 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3773 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3774 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3775 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3776 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3777 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3778 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3779 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3780 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3781 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3782 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3783 saved to "bulk_res


Row 3938 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3939 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3940 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3941 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3942 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3943 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3944 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3945 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3946 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3947 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3948 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3949 saved to "bulk_results" in MongoDB
-----------------------------------

Row 3950 saved to "bulk_res


Row 4095 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4096 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4097 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4098 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4099 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4100 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4101 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4102 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4103 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4104 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4105 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4106 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4107 saved to "bulk_res


Row 4259 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4260 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4261 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4262 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4263 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4264 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4265 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4266 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4267 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4268 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4269 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4270 saved to "bulk_results" in MongoDB
-----------------------------------

Row 4271 saved to "bulk_res

KeyError: 4401

In [56]:
# database = query_database()
# collection = query_collections(database)
# documents = query_documents(database,collection)


documents = query_documents('midland_property_info','addresses')
display(documents)

7 Databases Found ... 

-address_midland

-admin

-commerce_db

-config

-jj_earnings

-local

-midland_property_info

------------------------
Which Database would you like to load? Use exact spelling.. midland_property_info

Database "midland_property_info" Successfully Loaded..

 1 collection(s) found in "midland_property_info"
------------------------

0-addresses

------------------------

Which collection would you like to load? addresses
Collection found, Loading Documents.
want to see all documents? enter 'all' in both inputs. 
Desired Field in documents to pull matching results? all
Desired Value? NOTE: this will not pull numbers unless edited all


[{'_id': ObjectId('61e5eaba68822cfcae1f3e80'),
  'owner': 'PIERSON GARY DON',
  'address_1': '5403 S COUNTY ROAD 1140MIDLAND, TX 79706-7128',
  'address_2': 'COUNTY RD 01140 5403 S',
  'city': 'MIDLAND',
  'state': 'TEXAS',
  'full_address': 'COUNTY RD 01140   5403   S MIDLAND, TEXAS',
  '2021_building_value': 94160.0,
  '2021_land_value': 12000.0,
  '2021_total_value': 106160.0,
  'land_acre': 1.0,
  'land_sqft': 43560.0,
  'est_tax': 844.42,
  'total_sqft': 1316.0,
  '$_per_building_sqft': 71.6,
  '$_per_total_value_sqft': 80.7},
 {'_id': ObjectId('61e5eaba68822cfcae1f3e81'),
  'owner': 'PORTER STEPHEN RAY',
  'address_1': '5401 S COUNTY RD 1140MIDLAND, TX 79706',
  'address_2': 'COUNTY RD 01140 5401 S',
  'city': 'MIDLAND',
  'state': 'TEXAS',
  'full_address': 'COUNTY RD 01140   5401   S MIDLAND, TEXAS',
  '2021_building_value': 55950.0,
  '2021_land_value': 58920.0,
  '2021_total_value': 114870.0,
  'land_acre': 4.91,
  'land_sqft': 213880.0,
  'est_tax': 1613.89,
  'total_sqft': 

In [57]:
os.getcwd()

'C:\\Users\\Alec\\Documents\\Programming_Resources\\Boot_Camp\\16_project03\\LOCAL\\IPYNB files'

In [58]:
data_no_dupes.to_csv('../clean_data/clean_home_no_dupes.csv')
data_no_dupes.to_json('../clean_data/clean_home_no_dupes.json')